# NORA FILALI SUBMISSION for Dubizzle - Data Science
This notebook contains all the steps that I went through to have a recommendation engine (including main visualisation steps except graphs)
The approach used is:
-  1-Hierarchical clustering using features engineering and information on categories, position, price, presence of       most queried words: This method enables to clusterise around 85% of listings (with each cluster containg less than 100 listings) 
- 2-Refining of the clustering with text classification using TF IDF: This enables to clusterise the remaining listings
- => To test the full approach and obtain suggestions for each listing, go straight to the last section (demo)

# <font color=blue>***Loading, pre-processing & visualisation of data***</font>

In [1]:
#Importing relevant packages
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.cluster import KMeans
from sklearn import metrics
import unicodedata
import sys
import seaborn as sns
sns.set(style="whitegrid", color_codes=True)
from kmodes.kmodes import KModes

In [2]:
#Loading data
input_queries=pd.read_csv("/Users/nora.filali/Downloads/olx_data_sample/za_queries_sample.csv",delimiter=',')
input_data=pd.read_csv("/Users/nora.filali/Downloads/olx_data_sample/za_sample_listings_incl_cat.csv",low_memory=False,delimiter=',')
input_data=input_data.drop(["Unnamed: 0"], axis=1)
input_queries=input_queries.drop(["Unnamed: 0"], axis=1)
input_data.head(5)

,item_id,seller_id,listing_title,listing_description,listing_price,category_sk,category_l1_name_en,category_l2_name_en,category_l3_name_en,listing_latitude,listing_longitude
0,0.0,0,Nice wooden makes,We build all different types for sale,17500.00,olx|mea|za|806|809,"Home, Garden & Tools",Garden & Braai,Unknown,-25.43067,27.84873
1,1.0,1,A Shinning 2013 Chevrolet 1.4 Utility Bakkie w...,A Stunning accident free bargain that has just...,94890.00,olx|mea|za|362|378|2012,Vehicles,Cars & Bakkies,Chevrolet,-29.73714,31.07364
2,2.0,2,Lampshades various,A variety of lampshades in white,20.00,olx|mea|za|806|807,"Home, Garden & Tools",Furniture & Decor,Unknown,-33.88159,18.55522
3,3.0,3,Toyota Corolla,"Toyota Corolla 1.3 Professional, Front Electri...",63995.00,olx|mea|za|362|378|2067,Vehicles,Cars & Bakkies,Toyota,-26.10757,28.05670
4,4.0,4,bench grinder and buffer,bench grinder and.buffer...R800 for both,800.00,olx|mea|za|806|910,"Home, Garden & Tools",Tools & DIY,Unknown,-26.17190,27.91318


In [3]:
input_data.info()
input_data.isnull().sum() #check for missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500016 entries, 0 to 500015
Data columns (total 11 columns):
item_id                500012 non-null float64
seller_id              500012 non-null object
listing_title          500012 non-null object
listing_description    500012 non-null object
listing_price          491890 non-null object
category_sk            500000 non-null object
category_l1_name_en    500000 non-null object
category_l2_name_en    499988 non-null object
category_l3_name_en    499988 non-null object
listing_latitude       499988 non-null float64
listing_longitude      499988 non-null float64
dtypes: float64(3), object(8)
memory usage: 42.0+ MB


item_id                   4
seller_id                 4
listing_title             4
listing_description       4
listing_price          8126
category_sk              16
category_l1_name_en      16
category_l2_name_en      28
category_l3_name_en      28
listing_latitude         28
listing_longitude        28
dtype: int64

In [4]:
input_data=input_data.dropna(how='any') #drop missing values as represent very low portion of data
input_data.info()
input_data.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 491878 entries, 0 to 500015
Data columns (total 11 columns):
item_id                491878 non-null float64
seller_id              491878 non-null object
listing_title          491878 non-null object
listing_description    491878 non-null object
listing_price          491878 non-null object
category_sk            491878 non-null object
category_l1_name_en    491878 non-null object
category_l2_name_en    491878 non-null object
category_l3_name_en    491878 non-null object
listing_latitude       491878 non-null float64
listing_longitude      491878 non-null float64
dtypes: float64(3), object(8)
memory usage: 45.0+ MB


,item_id,listing_latitude,listing_longitude
count,491878.000000,491878.000000,491878.000000
mean,249999.543503,-27.918195,27.168319
std,144359.674062,2.961778,3.408192
min,0.000000,-34.816300,16.900450
25%,124964.250000,-29.702790,27.648030
50%,249984.500000,-26.239060,28.087390
75%,375021.750000,-26.072610,28.322820
max,499999.000000,-22.381250,32.757660


In [5]:
for column in input_data.columns: 
    if not (column=="item_id") and not (column=="listing_title") and not (column=="listing_description"):
        df = input_data.groupby(column)['item_id'].nunique()
        print ("/ number of",column, len(df))

/ number of seller_id 152580
/ number of listing_price 9510
/ number of category_sk 356
/ number of category_l1_name_en 13
/ number of category_l2_name_en 55
/ number of category_l3_name_en 271
/ number of listing_latitude 1812
/ number of listing_longitude 1508


# <font color=blue>***Features engineering***</font>

In [6]:
#Loading most frequent queries to see if they are present in the title or the description of the listings
input_queries_restricted=input_queries[input_queries["cnt"]>=4000]
list_queries=input_queries_restricted["search_term"].tolist()
list_queries=[query.lower() for query in list_queries if len(query)>1]
list_queries.sort(key = len,reverse=True)
len(list_queries)

820

In [7]:
def which_query_contained(text):
    """
    Will look for the query in the text and return it if found
    """
    text=text.lower()
    found=0
    for query in list_queries:
        if query in text:
            found=1
            return query
    if found==0:
        return ""

In [8]:
#normalisation of position (to closest multiple of 2) and price (to closest power of 10) features to ease the clustering
input_data["listing_latitude"]=input_data["listing_latitude"].apply(lambda x:str(int(2 * round(float(x)/2))))
input_data["listing_longitude"]=input_data["listing_longitude"].apply(lambda x:str(int(2 * round(float(x)/2))))
input_data["listing_price_normalized"]=input_data["listing_price"].apply(lambda x:str(10**round(np.log10(int(round(float(x)))) - np.log10(5.5) + 0.5)))
input_data["queries"]=input_data["listing_title"].apply(lambda x: which_query_contained(x))
input_data["queries_description"]=input_data["listing_description"].apply(lambda x: which_query_contained(x))

#Creation of categories level based on the position, price, and presence of a most demanded query in the title or description of document 
input_data["listing_category_position"]=input_data["category_sk"]+"-"+input_data["listing_latitude"]+"-"+input_data["listing_longitude"]
input_data["listing_category_position_price"]=input_data["listing_category_position"]+"-"+input_data["listing_price_normalized"]
input_data["listing_category_position_price_queries"]=input_data["listing_category_position_price"]+"-"+input_data["queries"]
input_data["listing_category_position_price_queries_desc"]=input_data["listing_category_position_price_queries"]+"-"+input_data["queries_description"]

In [9]:
#Visualising Data after features engineering
for column in input_data.columns: 
    if not (column=="item_id") and not (column=="listing_title") and not (column=="listing_description")and not (column=="seller_id")and not (column=="listing_price"):
        df = input_data.groupby(column)['item_id'].nunique()
        print ("/ number of",column, len(df))

/ number of category_sk 356
/ number of category_l1_name_en 13
/ number of category_l2_name_en 55
/ number of category_l3_name_en 271
/ number of listing_latitude 7
/ number of listing_longitude 9
/ number of listing_price_normalized 15
/ number of queries 809
/ number of queries_description 809
/ number of listing_category_position 3839
/ number of listing_category_position_price 11378
/ number of listing_category_position_price_queries 69244
/ number of listing_category_position_price_queries_desc 165429


# <font color=blue>***Gross clustering based on a hierarchical classification using categorization, position, price of listings and presence of a most queried string in it***</font>

In [10]:
#Generating dictionnaries for each category based on number of items in it:
df1=input_data.groupby('category_l1_name_en')['item_id'].nunique()
df1=df1.reset_index()
dico_l1=dict()
for i,row in df1.iterrows():
    temp={row["category_l1_name_en"]:row["item_id"]}
    dico_l1.update(temp)
    
df2=input_data.groupby('category_l2_name_en')['item_id'].nunique()
df2=df2.reset_index()
dico_l2=dict()
for i,row in df2.iterrows():
    temp={row["category_l2_name_en"]:row["item_id"]}
    dico_l2.update(temp)
    
df3=input_data.groupby('category_l3_name_en')['item_id'].nunique()
df3=df3.reset_index()
dico_l3=dict()
for i,row in df3.iterrows():
    temp={row["category_l3_name_en"]:row["item_id"]}
    dico_l3.update(temp)
    
df_cat_pos=input_data.groupby('listing_category_position')['item_id'].nunique()
df_cat_pos=df_cat_pos.reset_index()
dico_df_cat_pos=dict()
for i,row in df_cat_pos.iterrows():
    temp={row["listing_category_position"]:row["item_id"]}
    dico_df_cat_pos.update(temp)

df_cat_pos_price=input_data.groupby('listing_category_position_price')['item_id'].nunique()
df_cat_pos_price=df_cat_pos_price.reset_index()
dico_df_cat_pos_price=dict()
for i,row in df_cat_pos_price.iterrows():
    temp={row["listing_category_position_price"]:row["item_id"]}
    dico_df_cat_pos_price.update(temp)

df_cat_pos_price_queries=input_data.groupby('listing_category_position_price_queries')['item_id'].nunique()
df_cat_pos_price_queries=df_cat_pos_price_queries.reset_index()
dico_df_cat_pos_price_queries=dict()
for i,row in df_cat_pos_price_queries.iterrows():
    temp={row["listing_category_position_price_queries"]:row["item_id"]}
    dico_df_cat_pos_price_queries.update(temp)

df_cat_pos_price_queries_desc=input_data.groupby('listing_category_position_price_queries_desc')['item_id'].nunique()
df_cat_pos_price_queries_desc=df_cat_pos_price_queries_desc.reset_index()
dico_df_cat_pos_price_queries_desc=dict()
for i,row in df_cat_pos_price_queries_desc.iterrows():
    temp={row["listing_category_position_price_queries_desc"]:row["item_id"]}
    dico_df_cat_pos_price_queries_desc.update(temp)

In [11]:
def hierarchical_cluster(category1,category2,category3,category_location,category_location_price,category_location_price_query,category_location_price_query_desc):
    """hierarchical clusterisation based on the category, location, position, price and presence of frequent query. A listing is considered clusterized if the cluster in which it is mapped contains less than 100 elements"""
    
    if dico_l1[category1]<100:
        return category1
    else:
        if dico_l2[category2]<100:
            return category2
        else: 
            if dico_l3[category3]<100:
                return category3
            else:
                if dico_df_cat_pos[category_location]<100:
                    return category_location
                else:
                    if dico_df_cat_pos_price[category_location_price]<100:
                        return category_location_price
                    else:
                        if dico_df_cat_pos_price_queries[category_location_price_query]<100 and (category_location_price_query!=""):                            
                            return category_location_price_query
                        else:
                            if dico_df_cat_pos_price_queries_desc[category_location_price_query_desc]<100:
                                return category_location_price_query_desc
                            else:
                                return "no_category"

In [12]:
input_data["hierarchical_cluster"]=input_data.apply(lambda row: hierarchical_cluster(row['category_l1_name_en'], row['category_l2_name_en'], row['category_l3_name_en'],row['listing_category_position'],row['listing_category_position_price'],row['listing_category_position_price_queries'],row['listing_category_position_price_queries_desc']), axis=1)

In [13]:
#Visualisation of remaining data to clusterize
df5=input_data.groupby('hierarchical_cluster')['item_id'].nunique()
df5=df5.reset_index()
dico_l5=dict()
for i,row in df5.iterrows():
    temp={row['hierarchical_cluster']:row["item_id"]}
    dico_l5.update(temp)
count=0
for k in dico_l5.keys():
    if dico_l5[k]>=100:
        print(k,dico_l5[k])
        count+=1
print(count)
print(len(dico_l5))

no_category 83698
1
56908


In [14]:
#looking for categories that do not constitute clusters as cntains more than 100 elements
remaining=input_data[input_data["hierarchical_cluster"]=="no_category"].groupby('listing_category_position_price_queries_desc')['item_id'].nunique()
remaining=remaining.reset_index()
dico_remaining=dict()
for i,row in remaining.iterrows():
    temp={row['listing_category_position_price_queries_desc']:row["item_id"]}
    dico_remaining.update(temp)
len(dico_remaining)

350

# <font color=blue>***Refining of the clustering using text classification (TF IDF) within the pre-defined categories***</font>

In [15]:
#Pre-processing for the text classification
input_data["text_data"]=input_data["listing_title"]+ " "+ input_data["listing_description"]
input_data.loc[:, "text_data"] = input_data["text_data"].str.lower()

#remove punctuation
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                      if unicodedata.category(chr(i)).startswith('P'))
def remove_punctuation(text):
    return text.translate(tbl)

input_data["text_data"]=input_data["text_data"].apply( lambda x: remove_punctuation(str(x)))

In [16]:
#Getting the clusters for each listing by looking for the most similar listings within the predefined categories
remaining_subset={}
for k in dico_remaining.keys():
    print(k)
    subset=input_data[input_data["listing_category_position_price_queries_desc"]==k]
    subset=subset.reset_index()
    tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df = 0, max_df=0.8, stop_words='english')
    tfidf_matrix = tf.fit_transform(subset['text_data'])
    cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
    results = {}
    for idx, row in subset.iterrows():
        similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
        similar_items = [(cosine_similarities[idx][i], subset['item_id'][i]) for i in similar_indices]
        # First item is the item itself, so remove it.
        # Each dictionary entry is like: [(1,2), (3,4)], with each tuple being (score, item_id)
        results[row['item_id']] = similar_items[1:]
    remaining_subset.update(results)

olx|mea|za|16|301--26-28-1000.0--
olx|mea|za|16|301--26-28-1000.0-room to rent-room to rent
olx|mea|za|16|363--26-28-1000.0--
olx|mea|za|16|363--26-28-1000.0-rent-kitchen
olx|mea|za|16|363--26-28-1000.0-rent-rent
olx|mea|za|16|367--26-28-100000.0-house-kitchen
olx|mea|za|185|211--26-28-100.0--
olx|mea|za|185|211--26-28-1000.0--
olx|mea|za|185|214--26-28-100.0--
olx|mea|za|185|214--26-28-1000.0--
olx|mea|za|185|214--26-28-10000.0--
olx|mea|za|185|214--34-18-1000.0--
olx|mea|za|185|243--26-28-1000.0-guitar-guitar
olx|mea|za|185|820--26-28-100.0--
olx|mea|za|191|207--26-28-1.0--
olx|mea|za|191|207--26-28-100.0--
olx|mea|za|191|207--26-28-100.0--rent
olx|mea|za|191|207--26-28-1000.0--
olx|mea|za|191|207--34-18-100.0--rent
olx|mea|za|191|372--26-28-100.0--bed
olx|mea|za|191|633--26-28-1.0--
olx|mea|za|191|633--26-28-10.0--
olx|mea|za|191|633--26-28-10.0--paving
olx|mea|za|191|633--26-28-10.0-paving-
olx|mea|za|191|633--26-28-100.0--
olx|mea|za|191|633--26-28-100.0--domestic
olx|mea|za|191|6

olx|mea|za|362|379|3051--26-28-10000.0-kawasaki-kawasaki
olx|mea|za|362|379|3090--26-28-10000.0-suzuki-suzuki
olx|mea|za|362|379|3090--26-28-100000.0-suzuki-bike
olx|mea|za|362|379|3101--26-28-10000.0-yamaha-yamaha
olx|mea|za|362|379|3104--26-28-1000.0-helmet-helmet
olx|mea|za|362|416|4082--26-28-100000.0-freightliner-freightliner
olx|mea|za|362|416|4133--26-28-100000.0-isuzu-isuzu
olx|mea|za|362|416|4191--26-28-100000.0-mercedes benz-mercedes benz
olx|mea|za|362|416|4191--26-28-100000.0-mercedes-mercedes
olx|mea|za|362|416|4209--26-28-100000.0-nissan-nissan
olx|mea|za|362|416|4274--26-28-100000.0-hino-hino
olx|mea|za|362|416|4302--26-28-100000.0-tipper-tipper
olx|mea|za|362|416|4302--26-28-100000.0-truck-truck
olx|mea|za|362|417|5169--26-28-1000.0-trailer-trailer
olx|mea|za|362|417|5169--26-28-10000.0-trailer-trailer
olx|mea|za|5170|5172--26-28-100.0--
olx|mea|za|5170|5172--26-28-1000.0--
olx|mea|za|5170|5172--26-28-1000.0-desk-desk
olx|mea|za|5170|5173--26-28-1000.0--
olx|mea|za|5170

olx|mea|za|815|819--26-28-10000.0--
olx|mea|za|815|819--30-32-100.0--
olx|mea|za|815|819--34-18-1000.0--
olx|mea|za|853|855--26-28-100.0--
olx|mea|za|853|856--26-28-100.0--
olx|mea|za|853|856--26-28-100.0-baby-baby
olx|mea|za|853|856--26-28-1000.0--
olx|mea|za|853|856--26-28-1000.0-baby-baby
olx|mea|za|881|882--26-28-1000.0--
olx|mea|za|881|882--26-28-1000.0-bicycle-bicycle
olx|mea|za|881|882--26-28-1000.0-bike-bike
olx|mea|za|881|882--26-28-1000.0-mountain bike-mountain bike
olx|mea|za|881|882--26-28-10000.0-mountain bike-bike
olx|mea|za|881|883--26-28-1000.0--
olx|mea|za|881|899--26-28-100.0--
olx|mea|za|881|899--26-28-1000.0--
olx|mea|za|881|899--26-28-1000.0--used


In [17]:
# Function definition to get a friendly item name from the description field, given an item ID
def item(id):
    return input_data[input_data["item_id"] == id]['listing_title'].tolist()[0]

# Just reads the results out of the dictionary. No real logic here.
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + " id:"+ str(item_id) +"...")
    print("-------")
    try: 
        recs = remaining_subset[item_id][:num]
        for rec in recs:
            print("Recommended: " + " id:"+ str(rec[1]) +" "+ item(rec[1]) + " (through tf idf score:" + str(rec[0]) + ")")
    except:
        category=input_data[input_data["item_id"] == item_id]["hierarchical_cluster"].tolist()[0]
        recs=input_data[input_data["hierarchical_cluster"] == category]['listing_title'].tolist()[:num+1]
        for rec in recs:
                print("Recommended: " + rec)


# <font color=blue>***Demonstration and prediction of the suggestions for a listing***</font>

In [18]:
recommend(item_id=9987, num=10)
# plug in any item id here, and the number of recommendations you want (1-99)

Recommending 10 products similar to Acer laptop for sale Windows 7 for sale now with original charger id:9987...
-------
Recommended:  id:132833.0 Laptop (through tf idf score:0.386005104295)
Recommended:  id:253517.0 Acer Laptop (through tf idf score:0.334438924967)
Recommended:  id:305824.0 Acer Laptop For Sale (through tf idf score:0.304111636942)
Recommended:  id:454147.0 ACER Laptop Windows 8.1 (through tf idf score:0.29586884813)
Recommended:  id:375045.0 Acer laptop for sale (through tf idf score:0.256263957496)
Recommended:  id:135302.0 laptop (through tf idf score:0.243761509898)
Recommended:  id:343973.0 Acer laptop (through tf idf score:0.231621070009)
Recommended:  id:877.0 Acer laptop (through tf idf score:0.228057029639)
Recommended:  id:69790.0 Acer i5 Laptop 8gb 500gb hdd for sale (through tf idf score:0.212416309869)
Recommended:  id:48949.0 Hp compaq laptop still in great condition (through tf idf score:0.199683957225)


In [19]:
input_data[input_data["item_id"]==9987]

,item_id,seller_id,listing_title,listing_description,listing_price,category_sk,category_l1_name_en,category_l2_name_en,category_l3_name_en,listing_latitude,listing_longitude,listing_price_normalized,queries,queries_description,listing_category_position,listing_category_position_price,listing_category_position_price_queries,listing_category_position_price_queries_desc,hierarchical_cluster,text_data
9987,9987.0,8474,Acer laptop for sale Windows 7 for sale now wi...,Acer laptop in great condition comes in a bag ...,1800.00,olx|mea|za|800|803,Electronics & Computers,Computers & Laptops,Unknown,-26,28,1000.0,laptop,laptop,olx|mea|za|800|803--26-28,olx|mea|za|800|803--26-28-1000.0,olx|mea|za|800|803--26-28-1000.0-laptop,olx|mea|za|800|803--26-28-1000.0-laptop-laptop,no_category,acer laptop for sale windows 7 for sale now wi...


In [20]:
input_data[input_data["item_id"]==132833]

,item_id,seller_id,listing_title,listing_description,listing_price,category_sk,category_l1_name_en,category_l2_name_en,category_l3_name_en,listing_latitude,listing_longitude,listing_price_normalized,queries,queries_description,listing_category_position,listing_category_position_price,listing_category_position_price_queries,listing_category_position_price_queries_desc,hierarchical_cluster,text_data
132834,132833.0,51615,Laptop,Acer laptop for sale,1800.00,olx|mea|za|800|803,Electronics & Computers,Computers & Laptops,Unknown,-26,28,1000.0,laptop,laptop,olx|mea|za|800|803--26-28,olx|mea|za|800|803--26-28-1000.0,olx|mea|za|800|803--26-28-1000.0-laptop,olx|mea|za|800|803--26-28-1000.0-laptop-laptop,no_category,laptop acer laptop for sale
